# Python для анализа данных

*Алла Тамбовцева, НИУ ВШЭ*

дополнения: *Ян Пиле, НИУ ВШЭ*

Посмотрим на другие примеры использования `selenium`. 

**Пример.** Зайдем на сайт книжного магазина и найдем все книги про Python. Загрузим библиотеку, веб-драйвер и откроем страницу в браузере через Python.

In [1]:
from selenium import webdriver as wb
br = wb.Firefox()

# открываем страницу в Chrome в автоматическом режиме
br.get("http://www.biblio-globus.ru/")

Найдем с помощью CSS Selector'а (*SelectorGadget*) поле для ввода названия книги или автора. 

In [2]:
field = br.find_element_by_xpath('//*[@id="search_string"]')

Сохраним запрос:

In [3]:
author = "Python"  # переменная author - условность

Введем запрос в поле для поиска (`.send_keys`) и подождем чуть-чуть:

In [4]:
field.send_keys(author)
br.implicitly_wait(2)  # подождем пару секунд

Теперь найдем кнопку для поиска (значок *лупа* рядом со строкой поиска) через CSS Selector:

In [5]:
submit = br.find_element_by_css_selector("#search_submit")

Кликнем на нее:

In [6]:
submit.click()

Сохраним первую страницу с результатами в переменную `page1`.

In [7]:
page1 = br.page_source

In [9]:
# page1

Теперь обработаем эту страницу через `BeautifulSoup`:

In [10]:
from bs4 import BeautifulSoup

In [11]:
soup1 = BeautifulSoup(page1, 'lxml')

Найдем все названия книг на этой странице. По исходному коду можно увидеть, что они имеют тэг `a` с атрибутом `class`, равным `name`:

In [12]:
soup1.find_all('a', {'class':'name'})

[<a class="name" href="/search/catalog/details/10874070">Искусственный интеллект и компьютерное зрение. Реальные проекты на Python, Keras и TensorFlow</a>,
 <a class="name" href="/search/catalog/details/10877910">Python. Исчерпывающее руководство</a>,
 <a class="name" href="/search/catalog/details/10877909">Знакомство с Python</a>,
 <a class="name" href="/search/catalog/details/10874913">Высокопроизводительные Python-приложения. Практическое руководство по эффективному программированию</a>,
 <a class="name" href="/search/catalog/details/10874770">Профессиональное программирование. Python 3 и PyQt 6. Разработка приложений</a>,
 <a class="name" href="/search/catalog/details/10879726">Глубокое обучение на Python</a>,
 <a class="name" href="/search/catalog/details/10878167">Миссия: Python. Создаем игры вместе с детьми</a>,
 <a class="name" href="/search/catalog/details/10796619">Python для детей и родителей.</a>,
 <a class="name" href="/search/catalog/details/10776656">Справочник PYTHON.  

С помощью списковых включений выберем из ссылок с тэгом `<a>` текст (так мы уже делали, и не раз).

In [36]:
books1 = [b.text for b in soup1.find_all('a', {'class':'name'})]

In [37]:
books1

['Искусственный интеллект и компьютерное зрение. Реальные проекты на Python, Keras и TensorFlow',
 'Python. Исчерпывающее руководство',
 'Знакомство с Python',
 'Высокопроизводительные Python-приложения. Практическое руководство по эффективному программированию',
 'Профессиональное программирование. Python 3 и PyQt 6. Разработка приложений',
 'Глубокое обучение на Python',
 'Миссия: Python. Создаем игры вместе с детьми',
 'Python для детей и родителей.',
 'Справочник PYTHON.  Кратко, быстро, под рукой',
 'Python: Искусственный интеллект, большие данные и облачные вычисления']

Теперь аналогичным образом сгрузим информацию об авторах:

In [38]:
authors1 = [a.text for a in soup1.find_all('div', {'class': 'author'})]

Сгрузим расположение:

In [39]:
place1 = [p.text for p in soup1.find_all('div', {'class':'placement'})]

In [40]:
place1

['Расположение в торговом зале: Уровень 1, зал № 07, секция 08, шкаф 85, полка 04',
 'Расположение в торговом зале: Уровень 1, зал № 07, секция 08, шкаф 96, полка 08',
 'Расположение в торговом зале: Уровень 1, зал № 07, секция 08, шкаф 96, полка 08',
 'Расположение в торговом зале: Уровень 1, зал № 07, секция 08, шкаф 96, полка 02',
 'Расположение в торговом зале: Уровень 1, зал № 07, секция 08, шкаф 96, полка 04',
 'Расположение в торговом зале: Уровень 1, зал № 07, секция 08, шкаф 96, полка 05',
 'Расположение в торговом зале: Уровень 1, зал № 07, секция 08, шкаф 96, полка 06',
 'Расположение в торговом зале: Уровень 1, зал № 07, секция 08, шкаф 87, полка 06',
 'Расположение в торговом зале: Уровень 1, зал № 07, секция 08, шкаф 96, полка 05',
 'Расположение в торговом зале: Уровень 1, зал № 07, секция 08, шкаф 96, полка 04']

И, конечно, цену:

In [41]:
import re
price1 = [float(re.search(r'[\d,]+', p.text).group().replace(',','.')) 
          for p in soup1.find_all('div', 
                                  {'class':'title_data price'})]

In [42]:
price1

[3739.0, 2219.0, 2119.0, 1809.0, 2079.0, 2789.0, 1589.0, 1219.0, 519.0, 3389.0]

Осталось пройтись по всем страницам, которые были выданы в результате поиска. Для примера перейдем на страницу 2 и на этом остановимся.

In [30]:
next_p = br.find_element_by_css_selector('.next_page')

In [31]:
next_p.click()

Проделаем то же самое, что и с первой страницей. По-хорошему нужно написать функцию, которая будет искать на странице названия книг, их расположение и цену. Но оставим это в качестве задания читателю :)

In [35]:
page2 = br.page_source
soup2 = BeautifulSoup(page2, 'lxml')
books2 = [b.text for b in soup2.find_all('a', {'class':'name'})]
author2 = [a.text for a in soup2.find_all('div', {'class': 'author'})]
place2 = [p.text for p in soup2.find_all('div', {'class':'placement'})]
price2 = [float(re.search(r'[\d,]+', p.text).group().replace(',','.')) 
          for p in soup2.find_all('div', 
                                  {'class':'title_data price'})]

Расширим списки результатов с первой страницы данными, полученными со второй страницы, используя метод `.extend()`.

In [43]:
books1.extend(books2)
authors1.extend(books2)
place1.extend(place2)
price1.extend(price2)

Осталось импортировать библиотеку `pandas` и создать датафрейм.

In [44]:
import pandas as pd

Для разнообразия создадим датафрейм не из списка списков, а из словаря. Ключами словаря будут названия столбцов в таблице, а значениями – списки с сохраненной информацией (названия книг, цены и проч.).

In [46]:
df = pd.DataFrame({'book': books1, 'author': authors1,
                   'placement': place1, 'price': price1})

In [47]:
df.head()

,book,author,placement,price
0,Искусственный интеллект и компьютерное зрение....,"А. Коул ,С. Ганджу , М. Казам","Расположение в торговом зале: Уровень 1, зал №...",3739.0
1,Python. Исчерпывающее руководство,Д. Бизли,"Расположение в торговом зале: Уровень 1, зал №...",2219.0
2,Знакомство с Python,"Д. Бейдер , Д. Эймос , Д. Яблонски , Ф. Хейслер","Расположение в торговом зале: Уровень 1, зал №...",2119.0
3,Высокопроизводительные Python-приложения. Прак...,"М. Горелик, Й. Освальд","Расположение в торговом зале: Уровень 1, зал №...",1809.0
4,Профессиональное программирование. Python 3 и ...,"Н. А. Прохоренок , В. А. Дронов","Расположение в торговом зале: Уровень 1, зал №...",2079.0


In [50]:
list('Python')

['P', 'y', 't', 'h', 'o', 'n']

In [49]:
df.sort_values(by = 'price', ascending = False)

,book,author,placement,price
0,Искусственный интеллект и компьютерное зрение....,"А. Коул ,С. Ганджу , М. Казам","Расположение в торговом зале: Уровень 1, зал №...",3739.0
9,"Python: Искусственный интеллект, большие данны...","П.Дейтел, Х.Дейтел","Расположение в торговом зале: Уровень 1, зал №...",3389.0
5,Глубокое обучение на Python,Ф. Шолле,"Расположение в торговом зале: Уровень 1, зал №...",2789.0
12,"Python и наука о данных для чайников, 2-е издание","Python и наука о данных для чайников, 2-е издание","Расположение в торговом зале: Уровень 1, зал №...",2699.0
1,Python. Исчерпывающее руководство,Д. Бизли,"Расположение в торговом зале: Уровень 1, зал №...",2219.0
19,Python для чайников,Python для чайников,"Расположение в торговом зале: Уровень 1, зал №...",2119.0
2,Знакомство с Python,"Д. Бейдер , Д. Эймос , Д. Яблонски , Ф. Хейслер","Расположение в торговом зале: Уровень 1, зал №...",2119.0
4,Профессиональное программирование. Python 3 и ...,"Н. А. Прохоренок , В. А. Дронов","Расположение в торговом зале: Уровень 1, зал №...",2079.0
3,Высокопроизводительные Python-приложения. Прак...,"М. Горелик, Й. Освальд","Расположение в торговом зале: Уровень 1, зал №...",1809.0
16,Статистические вычисления на платформе Jupyter...,Статистические вычисления на платформе Jupyter...,"Расположение в торговом зале: Уровень 1, зал №...",1689.0


Давайте приведем столбец с ценой к числовому типу. Уберем слова *Цена* и *руб*, а потом сконвертируем строки в числа с плавающей точкой. Напишем функцию `get_price()`,

In [26]:
def get_price(price):
    book_price = price.split(' ')[1]  # разобьем строку по пробелу и возьмем второй элемент
    book_price = book_price.replace(',', '.')  # заменим запятую на точку
    price_num = float(book_price)  # сконвертируем в float
    return price_num

In [31]:
import re
def preis(x):
    return float('.'.join(re.findall(r'\d+',x)))

In [32]:
# проверка
get_price(df.price[0])

739.0

In [33]:
preis(df.price[0])

739.0

Всё отлично работает! Применим функцию к столбцу *price* и создадим новый столбец *nprice*.

In [34]:
df['nprice'] = df.price.apply(preis)

In [35]:
df.head()

,book,author,placement,price,nprice
0,Легкий способ выучить Python 3 еще глубже,З. Шоу,"Расположение в торговом зале: Уровень 1, зал №...","Цена: 739,00 руб.",739.0
1,Сумка-шоппер ErichKrause® 10L Python Print,,"Расположение в торговом зале: Уровень 1, зал №...","Цена: 419,00 руб.",419.0
2,"Изучаем Python, том 2,",М.Лутц,"Расположение в торговом зале: Уровень 1, зал №...","Цена: 2769,00 руб.",2769.0
3,"Изучаем Python: программирование игр, визуализ...",Э. Мэтиз,"Расположение в торговом зале: Уровень 1, зал №...","Цена: 1509,00 руб.",1509.0
4,Глубокое обучение и TensorFlow для профессиона...,С. Паттанаяк,"Расположение в торговом зале: Уровень 1, зал №...","Цена: 1659,00 руб.",1659.0


Теперь можем расположить книги по цене в порядке возрастания:

In [36]:
df.sort_values('nprice')

,book,author,placement,price,nprice
1,Сумка-шоппер ErichKrause® 10L Python Print,,"Расположение в торговом зале: Уровень 1, зал №...","Цена: 419,00 руб.",419.0
9,Легкий способ выучить Python,З. Шоу,"Расположение в торговом зале: Уровень 1, зал №...","Цена: 699,00 руб.",699.0
13,Легкий способ выучить Python 3,Легкий способ выучить Python 3,"Расположение в торговом зале: Уровень 1, зал №...","Цена: 739,00 руб.",739.0
0,Легкий способ выучить Python 3 еще глубже,З. Шоу,"Расположение в торговом зале: Уровень 1, зал №...","Цена: 739,00 руб.",739.0
10,Python для детей : самоучитель по программиров...,Python для детей : самоучитель по программиров...,"Расположение в торговом зале: Уровень 1, зал №...","Цена: 1069,00 руб.",1069.0
17,Python 3 Самое необходимое,Python 3 Самое необходимое,"Расположение в торговом зале: Уровень 1, зал №...","Цена: 1139,00 руб.",1139.0
12,Классические задачи Computer Science на языке ...,Классические задачи Computer Science на языке ...,"Расположение в торговом зале: Уровень 1, зал №...","Цена: 1219,00 руб.",1219.0
18,Глубокое обучение на Python,Глубокое обучение на Python,"Расположение в торговом зале: Уровень 1, зал №...","Цена: 1229,00 руб.",1229.0
14,Прикладной анализ текстовых данных на Python. ...,Прикладной анализ текстовых данных на Python. ...,"Расположение в торговом зале: Уровень 1, зал №...","Цена: 1309,00 руб.",1309.0
7,Изучаем программирование на Python,П. Бэрри,"Расположение в торговом зале: Уровень 1, зал №...","Цена: 1359,00 руб.",1359.0


И сохраним всю таблицу в csv-файл:

In [ ]:
df.to_csv("books.csv")